In [13]:
import os
import Bio.PDB
import pandas as pd
path_csv = '/home/tongwade780/pdb_website/onebyone_cluster_code/dssp/csv'
dssp_exec ='/opt/dssp/dssp'
pdb_path = '/home/tongwade780/pdb_website/onebyone_cluster_code/muscle_alignment_pdb'
idx = pd.IndexSlice
def getDSSP(PDBPath):
    idx = pd.IndexSlice
    
    # Use DSSP to assign secondary structure type to each residue.
    pdb_parser = Bio.PDB.PDBParser()
    structure = pdb_parser.get_structure('PDB', PDBPath)
    model = structure[0]
    dssp_results = Bio.PDB.DSSP(model, PDBPath, dssp=dssp_exec)

    chain_ID_resid_list = dssp_results.keys()
    dssp_results_table = pd.DataFrame(list(dssp_results))
    dssp_results_table = dssp_results_table[[0,1,2,3,4,5]]
    dssp_results_table.columns = ['DSSP Index', 'Residue Type', 'SS', 'Acc', 'Phi', 'Psi']
    dssp_results_table.index = pd.MultiIndex.from_tuples([(chain_ID_resid_list[x][0], chain_ID_resid_list[x][1][1]) for x in range(len(chain_ID_resid_list))])
    dssp_results_table = dssp_results_table.reindex()

    # Make sure all helices have negative phi and psi angles, otherwise change them to '-'
    helix_only_rows = (dssp_results_table['SS'] == 'H')
    helix_only_table = dssp_results_table[helix_only_rows]
    pos_phi_or_psi_idx = helix_only_table[(helix_only_table['Phi'] >= 0).values + (helix_only_table['Psi'] >= 0).values].index.tolist()
    dssp_results_table.loc[idx[:,pos_phi_or_psi_idx], 'SS'] = '-'
    return dssp_results_table, dssp_results



In [14]:
import os
import csv
pdb_path = '/mnt/Tsunami_HHD/Tang/onebyone_clsuter/muscle_alignment_pdb/'
pdbname_file = open ('/home/tongwade780/pdb_website/all_ca_number_all_code/onechain_100clust.csv').readlines ()
next_file = []
for line in pdbname_file[0:]:
    next_file.append(line.split(','))
cluster_1 = next_file[0]
#pdbname = cluster_1[0][0:4].lower()    
pdbname = '1bv3'
pdb_file = '{0}{1}'.format(pdbname,'.pdb')
pdbPATH = os.path.join(pdb_path,pdb_file)
[dssp_results_table,dssp_results] = getDSSP(pdbPATH)

In [12]:
len(pd.DataFrame(list(dssp_results)))

256

In [52]:
a = dssp_results_table['Residue Type']

In [3]:
secondary2score = {'H':0.5,'I':0.5,'G':0.5,'T':1,'S':1,'B':0,'E':0}
assigneds = secondary2score.keys()
secondaryScore = dssp_results_table['Residue Type'].apply(lambda x: secondary2score[x] if x in assigneds else 1)
final_table = pd.DataFrame(zip(dssp_results_table['DSSP Index'],secondaryScore),columns=['RESid','dssp_result'])

In [3]:
dssp_results_table

DSSP Index Residue Type SS       Acc    Phi    Psi
A 2             1            K  -  0.390244  360.0  134.9
  3             2            V  B  0.669014 -103.0  111.4
  4             3            F  -  0.081218  -74.5  148.7
  5             4            G  -  0.404762  -74.6  157.2
  6             5            R  H  0.346774  -55.4  -54.0
  7             6            C  H  0.333333  -69.3  -32.8
  8             7            E  H  0.448454  -66.0  -39.5
  9             8            L  H  0.000000  -69.7  -40.7
  10            9            A  H  0.037736  -54.2  -44.7
  11           10            A  H  0.500000  -63.3  -43.6
  12           11            A  H  0.122642  -64.7  -44.6
  13           12            M  H  0.005319  -62.8  -39.6
  14           13            K  H  0.624390  -62.8  -44.0
  15           14            R  H  0.745968  -62.4  -36.7
  16           15            H  T  0.163043  -86.4    9.6
  17           16            G  T  0.416667   78.7   22.2
  18           17            L  -  0.000000  -79.0  -29.0
  19           18            D  T  0.312883  -59.7  120.8
  20           19            N  T  0.617834   64.2   19.4
  21           20            Y  B  0.328829  -69.1  130.1
  22           21            R  T  0.588710   47.7   35.6
  23           22            G  T  0.797619   82.1    3.7
  24           23            Y  B  0.202703 -100.0  114.1
  25           24            S  -  0.284615  -63.4  154.9
  26           25            L  H  0.048780  -48.9  -38.6
  27           26            G  H  0.011905  -56.3  -35.3
  28           27            N  H  0.127389  -59.1  -47.1
  29           28            W  H  0.000000  -67.5  -42.7
  30           29            V  H  0.000000  -63.7  -45.0
  31           30            C  H  0.000000  -60.0  -47.0
...           ...          ... ..       ...    ...    ...
  100          99            V  H  0.000000  -75.9  -17.0
  101         100            S  T  0.300000  -81.6   -5.5
  102         101            D  S  0.631902  -80.7   -9.7
  103         102            G  S  0.690476  128.9  -37.6
  104         103            N  S  0.738854  -97.9    7.9
  105         104            G  G  0.107143   59.3 -141.0
  106         105            M  G  0.000000  -78.0   -5.0
  107         106            N  G  0.299363  -61.4  -16.4
  108         107            A  G  0.500000  -52.1  -32.1
  109         108            W  S  0.039648  -95.9  107.0
  110         109            V  H  0.549296  -58.3  -38.9
  111         110            A  H  0.132075  -64.9  -35.8
  112         111            W  H  0.052863  -64.0  -49.6
  113         112            R  H  0.419355  -57.9  -42.7
  114         113            N  H  0.649682  -88.9  -20.2
  115         114            R  H  0.544355 -127.8  -15.0
  116         115            C  T  0.000000 -119.9  -46.1
  117         116            K  T  0.502439  -49.3  126.2
  118         117            G  T  1.000000   84.6   -2.3
  119         118            T  S  0.478873 -100.4  175.9
  120         119            D  -  0.582822  -79.2   78.5
  121         120            V  G  0.112676  -69.2  -20.7
  122         121            Q  G  0.419192  -60.0  -28.5
  123         122            A  G  0.320755  -49.7  -34.7
  124         123            W  G  0.233480  -66.2  -19.0
  125         124            I  G  0.224852 -116.4    7.7
  126         125            R  T  0.487903  -70.1  134.7
  127         126            G  T  0.964286   80.6  -12.8
  128         127            C  -  0.266667  -76.5  142.9
  129         128            R  -  1.000000  -84.1  360.0

[128 rows x 6 columns]

In [70]:
final_table.to_csv(os.path.join(path_csv,pdbname+'_dssp.csv'),index = None)

In [71]:
pdbname

'5p6u'